# Poland 2D Line 001 drop auxiliary channels
This example segy file has auxiliary channels, i.e. traces that do not map to the field acquisition geometry, but intended for other purposes. To drop those traces, this example creates a new bytearray object and assigns byte data if the trace number within ensemble (often referred to as "channel") is greater than zero. The aux channels in this segy file have a value of -1.

This example uses the open source 2D land vibroseis line from [SEG Wiki](https://wiki.seg.org/wiki/2D_Vibroseis_Line_001).

In [1]:
import os
import io
import pathlib
from poland2d_context import segytools

TEXTENCODE = 'ebcdic'
BYTEORDER = '>'

def drop_aux_channels(input_segy_file:str, output_segy_context_manager:io.BufferedReader):
    path_segy_file = pathlib.Path(str(input_segy_file))
    assert (path_segy_file.is_file())
    print(f"reading {path_segy_file.name}")

    segy_file_size = os.path.getsize(input_segy_file)
    print(f"segy file size {segy_file_size}")
    
    file_header_input = segytools.SegyFileHeaderRev2(segy_logger=None)
    trace_header_input = segytools.SegyTraceHeaderRev2(segy_logger=None)
    
    # 'rb' is "read bytes"
    with open(path_segy_file, 'rb') as fobj:
        # read the first 3200 bytes.
        # This will always be 3200 byte textual file header
        b_text_header = fobj.read(3200)
        
        b_file_header = fobj.read(file_header_input.byte_length)
        file_header_input.set_header_values(buf=b_file_header, byteorder=BYTEORDER)
        
        sample_size_in_bytes = file_header_input.sample_format_size_in_bytes()
        trc_data_length_in_bytes = file_header_input.num_samples_per_trace.value * sample_size_in_bytes
    
        output_segy_context_manager.write(b_text_header)
        output_segy_context_manager.write(b_file_header)
        
        # Loop through traces ...
        while fobj.tell() < segy_file_size:
            # TRACE HEADER
            b_trace_header = fobj.read(trace_header_input.byte_length)
            trace_header_input.set_header_values(buf=b_trace_header, byteorder=BYTEORDER)
            if trace_header_input.num_samples.value != file_header_input.num_samples_per_trace.value:
                trc_data_length_in_bytes = file_header_input.num_samples_per_trace.value * sample_size_in_bytes
            # TRACE DATA
            b_trace_data = fobj.read(trc_data_length_in_bytes)
            # In this example segy file, all 'live' traces have the field 
            # 'trc_num_within_field_record' values greater than zero
            if trace_header_input.trc_num_within_field_record.value > 0:
                output_segy_context_manager.write(b_trace_header)
                output_segy_context_manager.write(b_trace_data)

        fobj.close()


path_segy_file_in = pathlib.Path.cwd().joinpath("data", "Line_001.sgy")
path_segy_file_out = pathlib.Path.cwd().joinpath("data", "Line_001_NO_AUX.sgy")
with open(path_segy_file_out, 'ab') as fout:
    drop_aux_channels(input_segy_file=str(path_segy_file_in), output_segy_context_manager=fout)
    fout.close()
  

reading Line_001.sgy
segy file size 445100896
